# Dose-Response Analysis

This notebook demonstrates dose-response curve fitting using the 4-parameter logistic (Hill) equation.

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from notebook_utils import get_api_client, demo_mode_banner

# Optional API integration (not required for the static demo below)
api_url = os.environ.get("API_URL") or os.environ.get("AMPRENTA_API_URL")
client, demo_mode = get_api_client(api_url=api_url)
if demo_mode or client is None:
    demo_mode_banner()

# Set style
plt.style.use('seaborn-v0_8')


## Example Dose-Response Data

In [ ]:
# Example dose-response data (concentrations in nM, response as % inhibition)
concentrations = np.array([0.1, 1, 10, 100, 1000, 10000, 100000])  # nM
response = np.array([5, 15, 35, 65, 85, 92, 95])  # % inhibition

df = pd.DataFrame({
    'concentration_nM': concentrations,
    'response_percent': response
})

print(df)
print(f"\nData range: {concentrations.min():.1f} - {concentrations.max():.1f} nM")

## 4-Parameter Logistic (Hill) Curve Fitting

In [ ]:
def hill_equation(x, bottom, top, ec50, hill_slope):
    """4-parameter logistic (Hill) equation.

    Parameters:
    - bottom: minimum response (baseline)
    - top: maximum response (plateau)
    - ec50: concentration at 50% response
    - hill_slope: Hill coefficient (steepness)
    """
    return bottom + (top - bottom) / (1 + (ec50 / x) ** hill_slope)

# Initial parameter guesses
initial_guess = [
    response.min(),  # bottom
    response.max(),  # top
    np.median(concentrations),  # ec50
    1.0  # hill_slope
]

# Fit the curve
popt, pcov = curve_fit(
    hill_equation,
    concentrations,
    response,
    p0=initial_guess,
    maxfev=10000
)

bottom, top, ec50, hill_slope = popt

print("Fitted parameters:")
print(f"  Bottom: {bottom:.2f}%")
print(f"  Top: {top:.2f}%")
print(f"  EC50: {ec50:.2f} nM")
print(f"  Hill slope: {hill_slope:.2f}")

## Calculate IC50/EC50

In [ ]:
# IC50 is the concentration at 50% inhibition
# For inhibition assays, IC50 = EC50 when baseline is 0 and top is 100
ic50 = ec50

print(f"IC50: {ic50:.2f} nM")
print(f"IC50: {ic50/1000:.4f} µM")
print(f"IC50: {ic50/1000000:.6f} mM")

In [ ]:
# --- Bayesian Dose-Response Fit ---
# Compare classical vs Bayesian with credible intervals

from amprenta_rag.analysis.bayesian_dose_response import fit_bayesian_dose_response

# Fit Bayesian model
bayes_result = fit_bayesian_dose_response(
    concentrations=concentrations.tolist() if hasattr(concentrations, "tolist") else list(concentrations),
    responses=response.tolist() if hasattr(response, "tolist") else list(response),
    likelihood="normal",
)

print(f"Bayesian EC50: {bayes_result['ec50_mean']:.4f} nM")
print(f"95% Credible Interval: {bayes_result['ec50_ci']}")
print(f"Hill Slope: {bayes_result['hill_slope']:.4f}")

# Plot comparison (classical curve + Bayesian credible band)
import numpy as np
import matplotlib.pyplot as plt

x_pred = np.logspace(np.log10(concentrations.min()), np.log10(concentrations.max()), 200)

# Classical prediction
y_classical = hill_equation(x_pred, bottom, top, ec50, hill_slope)

# Bayesian prediction band from posterior samples
trace = bayes_result["trace"]
post = trace.posterior

bottom_s = post["bottom"].values.reshape(-1)
top_s = post["top"].values.reshape(-1)
ec50_s = post["ec50"].values.reshape(-1)
hill_s = post["hill_slope"].values.reshape(-1)

# Limit samples for speed in template notebooks
n_samp = min(2000, bottom_s.shape[0])
idx = np.random.default_rng(0).choice(bottom_s.shape[0], size=n_samp, replace=False)

bottom_s = bottom_s[idx]
top_s = top_s[idx]
ec50_s = ec50_s[idx]
hill_s = hill_s[idx]

# 4PL: bottom + (top-bottom)/(1 + (x/ec50)^(-hill))
# (same as in bayesian_dose_response.py)
mu = bottom_s[:, None] + (top_s - bottom_s)[:, None] / (1.0 + (x_pred[None, :] / ec50_s[:, None]) ** (-hill_s[:, None]))

band_lo = np.quantile(mu, 0.025, axis=0)
band_hi = np.quantile(mu, 0.975, axis=0)
mu_mean = np.mean(mu, axis=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(concentrations, response, label="Data", zorder=3)
ax.plot(x_pred, y_classical, label="Classical fit", linewidth=2)
ax.plot(x_pred, mu_mean, label="Bayesian mean", linewidth=2)
ax.fill_between(x_pred, band_lo, band_hi, alpha=0.2, label="Bayesian 95% CI")

ax.set_xscale("log")
ax.set_xlabel("Concentration (nM)")
ax.set_ylabel("Response (% inhibition)")
ax.set_title("Classical vs Bayesian Dose-Response")
ax.legend()
plt.show()



In [ ]:
# --- Convergence Diagnostics ---
import arviz as az

# Trace plot
az.plot_trace(bayes_result["trace"], var_names=["ec50", "hill_slope"])
plt.tight_layout()
plt.show()

# Summary with R-hat and ESS
print(az.summary(bayes_result["trace"], var_names=["ec50", "hill_slope"]))



## Plot Dose-Response Curve

In [ ]:
# Generate smooth curve for plotting
x_fit = np.logspace(
    np.log10(concentrations.min()),
    np.log10(concentrations.max()),
    1000
)
y_fit = hill_equation(x_fit, bottom, top, ec50, hill_slope)

# Plot
plt.figure(figsize=(10, 6))
plt.semilogx(concentrations, response, 'o', markersize=8, label='Data', color='blue')
plt.semilogx(x_fit, y_fit, '-', label='Fitted curve', color='red', linewidth=2)

# Mark IC50
plt.axvline(ic50, color='green', linestyle='--', linewidth=2, label=f'IC50 = {ic50:.2f} nM')
plt.axhline(50, color='gray', linestyle=':', alpha=0.5)

plt.xlabel('Concentration (nM)', fontsize=12)
plt.ylabel('Response (% inhibition)', fontsize=12)
plt.title('Dose-Response Curve', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()